![Logo](https://docs.openmc.org/en/latest/_static/openmc_logo.png)

# Importing necessary libraries

In [ ]:
import os
import openmc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

# Necessary inputs

In [ ]:
# Statepoint file path:

path = os.getcwd() + '/statepoint.130.h5'

# Fuel assembly pitch:

pitch = 1.233598

# Number of rods in each dimension of the assembly:

N = 16

# Number of assemblies in each dimension of the core lattice:

NA = 13

# Outer radius of the core barrel:

TR = 167.64 + 16.83

# Model height:

height = 1.0

# Extracting data from the statepoint file

In [ ]:
# Opening the statepoint file (instatiates a StatePoint class object):

SP = openmc.StatePoint(path)

# Method to extract the tallies from the statepoint (returns a Tally class object):

universe_results = SP.get_tally(name = 'Universe Tally')
print(universe_results)

energy_results = SP.get_tally(name = 'Energy Tally')
print(energy_results)

assembly_mesh_results = SP.get_tally(name = 'Assembly Mesh Tally')
print(assembly_mesh_results)

rod_mesh_results = SP.get_tally(name = 'Rod Mesh Tally')
print(rod_mesh_results)

# Verifying source convergence

In [ ]:
keff = SP.k_generation
entropy = SP.entropy

CONV1 = plt.figure(figsize = (8, 6))
GCONV1 = CONV1.add_subplot(111)
GCONV1.plot(keff, '-r')
GCONV1.set_title('Convergence of $k_{eff}$', math_fontfamily = 'cm', size = 16, pad = 12)
GCONV1.set_xlabel('Cicle', size = 12)
GCONV1.set_ylabel('$k_{eff}$', math_fontfamily = 'cm', size = 12)

CONV2 = plt.figure(figsize = (8, 6))
GCONV2 = CONV2.add_subplot(111)
GCONV2.plot(entropy, '-g')
GCONV2.set_title('Convergence of Source Distribuition', size = 16, pad = 12)
GCONV2.set_xlabel('Cicle', size = 12)
GCONV2.set_ylabel('Shannon Entropy (bits)', size = 12)

# Source Distribuition

## Fission neutron energy

In [ ]:
# Energy bins from 1 keV to 10 MeV in logarithmic scale:

energy_bins = np.logspace(start = 3, stop = 7, num = 100, endpoint = True, base = 10)

# Plot the histogram:

SOURCE1 = plt.figure(figsize = (8, 6))
GSOURCE1 = SOURCE1.add_subplot(111, xscale = 'log')
values, bins, patches = GSOURCE1.hist(SP.source['E'], energy_bins, density = False)
GSOURCE1.set_title('Fission Neutron Energy', size = 16, pad = 12)
GSOURCE1.set_xlabel('Energy (eV)', math_fontfamily = 'cm', size = 12)
GSOURCE1.set_ylabel('Number of particles', math_fontfamily = 'cm', size = 12)

# Make sure that summing the values of all bins returns the total number of particles simulated:

print(sum(values))

## Fission locations

In [ ]:
SOURCE2 = plt.figure(figsize = (8, 6))
GSOURCE2 = SOURCE2.add_subplot(111)
VECTORS = GSOURCE2.quiver(SP.source['r']['x'], SP.source['r']['y'],
              SP.source['u']['x'], SP.source['u']['y'], 
              np.log10(SP.source['E']), cmap = 'jet', scale = 30)
SOURCE2.colorbar(VECTORS)
GSOURCE2.set_title('Fission Locations', size = 16, pad = 12)
GSOURCE2.set_xlim(-TR, TR)
GSOURCE2.set_ylim(-TR, TR)

## Delayed neutrons

In [ ]:
delayed_neutrons = []
for i in SP.source['delayed_group']:
    if i != 0:
        delayed_neutrons.append(i)

SOURCE3 = plt.figure(figsize = (8, 6))
GSOURCE3 = SOURCE3.add_subplot(111)
GSOURCE3.hist(delayed_neutrons, [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])
GSOURCE3.set_title('Delayed Neutrons', size = 16, pad = 12)
GSOURCE3.set_xlabel('Delayed Neutron Groups', size = 12)
GSOURCE3.set_ylabel('Number of particles', size = 12)
GSOURCE3.set_xticks([1, 2, 3, 4, 5, 6])

# Tally Analysis

## Normalization factor

In [ ]:
# Total volume of the model [m³]:

total_volume = np.pi*(TR**2)*height/1000000

# Estimated power density [W/m³]:

power_density = 107.9*1000*1000

# Estimated total power [W]:

power = power_density*total_volume

print(power)

# Conversion factor from eV to Joule [J/eV]:

eV = 1.602*(10**(-19))

# Value of 'heating-local' score over the entire model [eV/particle]:

H = universe_results.get_values(scores = ['heating-local'], filters = [], filter_bins = [], 
                                nuclides = [], value = 'mean')

print(H)

# Total heating [J/particle]:

H = H[0][0][0]*eV

print(H)

# Normalization factor [particle/second]:

Nfactor = power/H

print(Nfactor)

## Universe tally

In [ ]:
# Print the universe tally results:

universe_results.get_pandas_dataframe()

## Neutron flux spectrum

In [ ]:
energy_bins = np.logspace(start = -3, stop = 7, num = 1000, endpoint = True, base = 10)
energy_values = (np.diff(energy_bins)/2) + energy_bins[:999]

flux_spectrum = energy_results.get_values(scores = ['flux'], filters = [], filter_bins = [], 
                                          nuclides = [], value = 'mean').flatten()

flux_spectrum_std_dev = energy_results.get_values(scores = ['flux'], filters = [], filter_bins = [], 
                                                  nuclides = [], value = 'std_dev').flatten()

SPECTRUM1 = plt.figure(figsize = (8, 6))
GSPECTRUM1 = SPECTRUM1.add_subplot(111)
GSPECTRUM1.loglog(energy_values, flux_spectrum[:]/np.diff(energy_bins), base = 10, linewidth = 1)
GSPECTRUM1.fill_between(energy_values, (flux_spectrum[:] - flux_spectrum_std_dev)/np.diff(energy_bins), (flux_spectrum[:] + flux_spectrum_std_dev)/np.diff(energy_bins), alpha = 0.3)
GSPECTRUM1.set_title('Neutron Flux Spectrum', size = 16, pad = 12)
GSPECTRUM1.set_xlabel('Energy (eV)', math_fontfamily = 'cm', size = 12)
GSPECTRUM1.set_ylabel('Flux $(cm^{-2}s^{-1})$', math_fontfamily = 'cm', size = 12)

In [ ]:
flux_spectrum_rel_unc = energy_results.get_values(scores = ['flux'], filters = [], filter_bins = [], 
                                                  nuclides = [], value = 'rel_err').flatten()

SPECTRUM2 = plt.figure(figsize = (8, 6))
GSPECTRUM2 = SPECTRUM2.add_subplot(111)
GSPECTRUM2.hist(flux_spectrum_rel_unc, bins = 100)
GSPECTRUM2.set_title('NFS Relative Uncertainty', size = 16, pad = 12)
GSPECTRUM2.set_xlabel('Relative Uncertainty', size = 12)
GSPECTRUM2.set_ylabel('Number of bins', size = 12)

## Neutron flux

In [ ]:
flux_mean = assembly_mesh_results.get_values(scores = ['flux'], filters = [], filter_bins = [], nuclides = [], value = 'mean').reshape(13, 13, 1)

flux_unc = assembly_mesh_results.get_values(scores = ['flux'], filters = [], filter_bins = [], nuclides = [], value = 'std_dev').reshape(13, 13, 1)

flux_rel_unc = assembly_mesh_results.get_values(scores = ['flux'], filters = [], filter_bins = [], nuclides = [], value = 'rel_err').flatten()

FLUX1 = plt.figure(figsize = (8, 6))
GFLUX1 = FLUX1.add_subplot(111)
GFLUX1.set_title('Relative Assembly Power', size = 16, pad = 12)
FLUX1.colorbar(GFLUX1.imshow(flux_mean[:,:,0], cmap = 'jet'))

FLUX2 = plt.figure(figsize = (8, 6))
GFLUX2 = FLUX2.add_subplot(111)
GFLUX2.set_title('Neutron Flux Uncertainty', size = 16, pad = 12)
FLUX2.colorbar(GFLUX2.imshow(flux_unc[:,:,0], cmap = 'jet'))

FLUX3 = plt.figure(figsize = (8, 6))
GFLUX3 = FLUX3.add_subplot(111)
GFLUX3.set_title('NF Relative Uncertainty', size = 16, pad = 12)
GFLUX3.hist(flux_rel_unc, bins = 100)

In [ ]:
flux_mean = rod_mesh_results.get_values(scores = ['heating-local'], filters = [], filter_bins = [], nuclides = [], value = 'mean').reshape(16*13, 16*13, 1)

flux_unc = rod_mesh_results.get_values(scores = ['heating-local'], filters = [], filter_bins = [], nuclides = [], value = 'std_dev').reshape(16*13, 16*13, 1)

flux_rel_unc = rod_mesh_results.get_values(scores = ['heating-local'], filters = [], filter_bins = [], nuclides = [], value = 'rel_err').flatten()

FLUX1 = plt.figure(figsize = (8, 6))
GFLUX1 = FLUX1.add_subplot(111)
GFLUX1.set_title('Relative Power', size = 16, pad = 12)
FLUX1.colorbar(GFLUX1.imshow(flux_mean[:,:,0], cmap = 'jet'))

FLUX2 = plt.figure(figsize = (8, 6))
GFLUX2 = FLUX2.add_subplot(111)
GFLUX2.set_title('Neutron Flux Uncertainty', size = 16, pad = 12)
FLUX2.colorbar(GFLUX2.imshow(flux_unc[:,:,0], cmap = 'jet'))

FLUX3 = plt.figure(figsize = (8, 6))
GFLUX3 = FLUX3.add_subplot(111)
GFLUX3.set_title('NF Relative Uncertainty', size = 16, pad = 12)
GFLUX3.hist(flux_rel_unc, bins = 100)